# Pre-Assembling

In [1]:
import pandas as pd
import numpy as np
import anndata as ad
import re
import csv

from scipy.sparse import coo_matrix
from scipy.io import mmwrite
from pathlib import Path

import sctoolbox
import sctoolbox.utils as utils
import sctoolbox.utils.assemblers as assembler

### --- Datei einlesen ---

In [ ]:
file_path_pat = "/mnt/workspace_stud/napkon_data/wp2_rna/out.txt"
file_path_gene = "/mnt/workspace_stud/napkon_data/wp2_rna/combined_rna_napkon_4_all.matrix.raw.anno"


path_mtx = str(Path.cwd()) # Aktueller Speicherort/Verzeichnis mit .mtx, barcodes.tsv und genes.tsv
print(path_mtx)

### 1. --- Patienten-Metadaten lesen und vorbereiten für Assemblierung ---

In [ ]:
pat_df = pd.read_csv(file_path_pat, decimal=",", sep="\t")
pat_df = pat_df.fillna(0)
pat_df.head()

In [ ]:
pat_df["rna project.run id"] # ID überprüfen

In [5]:
# optional: Patienten-Datei im lokalen Speicherort abspeichern
# pat_df.to_csv('pat_meta.csv', index=False) 

### 2. --- Gen-Metadaten lesen und vorbereiten für Assemblierung ---

In [ ]:
data = []

with open(file_path_gene, 'r') as file:
    content = file.read()
    lines = content.splitlines()

    header = lines[0].split('\t')

    for line in lines[1:]:
        elements = line.split('\t')
        data.append(elements)

gene_df = pd.DataFrame(data, columns=header)
gene_df.head()

In [ ]:
gene_df.fillna(0)
gene_df.head()

In [8]:
gene_df.to_csv('gene_meta.csv', index =False) # Gen-Datei speichern

In [ ]:
features = gene_df.loc[:,['Ensembl gene id', 'Ensembl gene', 'Ensembl biotype', 'UniProt proteins', 'UniProt genes', 'UniProt accessions', 'UniProt names', 'UniProt Ensembl transcripts', 'UniProt Ensembl proteins', 'UniProt Ensembl gene ids', 'Ensembl chr', 'Ensembl start', 'Ensembl stop', 'Ensembl strand', 'KEGG PATHWAY terms', 'KEGG PATHWAY ids', 'Gene Ontology terms', 'Gene Ontology ids']] # Auswahl der ersten 18 Spalten -> die Gene ids und Spalten mit Metadaten
features.head()

In [10]:
features.to_csv("variables.tsv", sep="\t", index=False, header=False) # Variables-Datei speichern

### 3. --- Barcodes Datei aus Patienten-Metadaten erstellen ---

In [ ]:
# Barcodes-Datei (Patienten + Metadaten)

patient_ids = gene_df.columns[18:]  # Patienten-Spalten starten ab Spalte 18 
patient_ids

In [ ]:
barcodes = pd.DataFrame({
    "rna project.run id": patient_ids # Benennung der Spalte in barcodes.tsv mit "rna project.run id"
}).merge(pat_df, left_on="rna project.run id", right_on="rna project.run id", how="left")  # Metadaten anfügen
barcodes.to_csv("barcodes.tsv", sep="\t", index=False, header=False)
barcodes.head()

In [13]:
barcodes = pd.read_csv("barcodes.tsv", decimal=".", sep="\t") # Barcodes-Datei speichern

### 4. --- Count-Matrix erstellen aus Gen-Metadaten ---

In [ ]:
counts = gene_df.iloc[:, 18:].fillna(0) # Zählwerte extrahieren und NaN durch 0 ersetzen
counts

In [ ]:
sparse_matrix = coo_matrix(counts.values) # Sparse-Matrix erstellen
sparse_matrix

In [16]:
mmwrite("matrix.mtx", sparse_matrix) # MTX-Datei speichern

### 5. --- Anndata-Objekt aus .mtx, barcodes.tsv und genes.tsv erstellen ---

In [17]:
mtx_tsv = 'matrix.mtx' # Datei, die die Counts enthält
barcodes_tsv = 'barcodes.tsv' # Datei, die Barcode-Informationen enthält
variables_tsv = 'variables.tsv' # Datei, die variable Informationen enthält

In [ ]:
print(pat_df.shape, barcodes.shape, gene_df.shape, features.shape) # Shape überprüfen

In [ ]:
if path_mtx:
    adata = assembler.from_mtx(path_mtx, mtx=mtx_tsv, barcodes=barcodes_tsv, variables=variables_tsv)

### 6. --- AnnData vorbereiten ---

In [ ]:
# Adata Obs einen Header hinzufügren
adata.obs_names.name = 'rna project.run id'
obs_columns_list = pat_df.columns.tolist()
obs_columns_list.remove('rna project.run id')
obs_columns_list.append('filename') # möglicherweise muss die komplette Spalte raus
obs_columns_list.append('rel_path') # möglicherweise muss die komplette Spalte raus
obs_header = obs_columns_list
adata.obs.columns = obs_header
adata

In [21]:
var_header = features.columns.tolist()
var_header.remove("Ensembl gene id")
adata.var_names.name = "Ensembl gene id"
adata.var.columns = var_header

In [ ]:
with pd.option_context('display.max_rows', 5,'display.max_columns', None):
    display(adata.obs)
    display(adata.var)

In [ ]:
pat_df.columns.tolist()

In [24]:
# .obs-Spaltennamen, die gelöscht werden sollen
drop_obs = []

# .obs-Spaltennamen, die geändert werden sollen. z.B. "old_name": "new_name"
# Beispiel: "old_name" = "rna usable sample: mapped reads > 2000000" -> Regex bevorzugt
rename_obs = {'rna reads used for further steps (after optional filters for multimap, duplicate, mitochondria, rrna)':'rna reads used after filters', 'rna usable sample: mapped reads > 2000000':'mapped reads > 2000000'} 

#Namen ändern mithilfe von Regex
replace_obs = {'unassigned':'unass.','due to':'',' ':'_','%':'pct',':':'_of','>=':'at_least','>':'over'}
replace_var = {' ':'_'}

# .var-Spaltennamen, die gelöscht werden sollen
drop_var = []

# .var-Spaltennamen, die geändert werden sollen. z.B. "old_name": "new_name"
rename_var = {}

In [ ]:
# Änderungen an .obs-Spaltennamen in eine Kopie absspeichern
obs = adata.obs.copy()

obs.drop(columns=drop_obs, inplace=True)
obs.rename(columns=rename_obs, errors='raise', inplace=True)

# Änderungen an .var-Spaltennamen in eine Kopie absspeichern
var = adata.var.copy()

var.drop(columns=drop_var, inplace=True)
var.rename(columns=rename_var, errors='raise', inplace=True)

# Anderungen vornehmen
for x, y in replace_obs.items():
    obs.columns= obs.columns.str.replace(x, y, regex=True)
    
for x, y in replace_var.items():
    var.columns= var.columns.str.replace(x, y, regex=True) 
    
# Änderungen an AnnData anwenden
adata.obs = obs
adata.var = var

In [ ]:
# Änderung dtype, wenn nicht automatisch übernommen
adata.obs["mapped_reads_over_2000000"] = adata.obs["mapped_reads_over_2000000"].astype("category")

### 7. --- AnnData-Objekt speichern ---

In [ ]:
display(adata) # Übersicht

In [ ]:
adata_output = "anndata_1.h5ad"
utils.adata.save_h5ad(adata, adata_output) # speichern

In [33]:
sctoolbox.settings.close_logfile()